In [39]:
!pip install gurobipy

You should consider upgrading via the 'C:\Users\jgz\anaconda3\python.exe -m pip install --upgrade pip' command.


In [40]:
import warnings

In [41]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [42]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [43]:
# Create dictionaries to capture factory supply limits, depot throughput limits, and customer demand.

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5
})

In [44]:
model = gp.Model('SupplyNetworkDesign')
flow = model.addVars(arcs, obj=cost, name="flow")

In [45]:
# Production capacity limits

factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")

In [46]:
# Customer demand

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer")

In [47]:
# Depot flow conservation

depots = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

In [48]:
# Depot throughput

depot_capacity = model.addConstrs((gp.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")

In [49]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0x3607c855
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.812500e+04   0.000000e+00      0s
       7    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.985000000e+05


In [50]:
product_flow = pd.DataFrame(columns = ['From', 'To', 'Flow'], dtype=object)

In [51]:
for arc in arcs:
    if flow[arc].x > 1e-6:
        product_flow = product_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x}, ignore_index=True)  
product_flow.index=[''] * len(product_flow)

In [52]:
product_flow

,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,55000.0
,Brighton,Exeter,40000.0
,Birmingham,C2,10000.0
,Birmingham,C4,35000.0
,Birmingham,C5,5000.0
,London,C5,55000.0
,Exeter,C3,40000.0
